In [ ]:
#limpio la memoria
rm(list=ls())
gc()

#cargo librerias
library("data.table")
library("ROCR")
library("rpart")
library("rpart.plot")
library("caret")

In [ ]:
setwd("~/cloudL/UA/labo2022/")

#Parametros entrada

#kcarpeta_datasets    <- "../input/laboratorio-de-implementacion-i-2021/"   #KAGGLE
kcarpeta_datasets    <- "./datasets/"                          #VM o Ubuntu

#Archivo con datos etiquetados para entrenamiento
karchivo_entrada      <-  paste0(kcarpeta_datasets, "competencia1_2022.csv")

#Formato para submit en Kaggle
#karchivo_score      <-  "../input/uamds2020ldi1f1/Sample_201910_Fase_I.txt"

#Separador de campos en archivos
kcampos_separador     <-  "\t"

#Campo que identifica las muestras
kcampo_id             <-  "numero_de_cliente"

#Campo que contiene la clase a estimar
kclase_nomcampo       <-  "clase_ternaria"

#Valor de interés
kclase_valor_positivo <-  "BAJA+2"

#Campos a borrar para el entrenamiento
kcampos_a_borrar      <-  c(kcampo_id,kclase_nomcampo,"foto_mes")

#Campo que contendrá a la variable objetivo generada
kobjetivo             <-  "clase"

#Identificación del modelo
kmodelo               <-  "02-RPART"

#Ganancia por TP
kTPGain               <-  78000

#Pérdida por FP
kFPGain               <-  -2000

#Establezco semilla aleatoria
set.seed(1)



In [ ]:
#cargo los datos
dataset <- fread(karchivo_entrada)

#Para hacer pruebas rapidas puedo reducir el dataset a una fraccion
#subsample <- sample(1:nrow(dataset), .1 * nrow(dataset))
#dataset <- dataset[subsample,]

dtrain  <- dataset[ foto_mes==202101 ]  #defino donde voy a entrenar

train_rows <- createDataPartition(dtrain$clase_ternaria, p = .66, list = FALSE)

#train_rows <- sample(1:nrow(dataset), .66 * nrow(dataset))
dtest <- dtrain[-train_rows,]
dtrain <- dtrain[train_rows,]

dapply  <- dataset[ foto_mes==202103 ]  #defino donde voy a aplicar el modelo

In [ ]:
#Funcion que calcula la ganancia para una prediccion y valores reales
fmetrica_ganancia_rpart  = function( probs, clases, pclase_valor_positivo )
{
 
  return(  sum(    (probs > 1/40 ) * 
                   ifelse( clases== pclase_valor_positivo, kTPGain, kFPGain )   
              )
         )
}

In [ ]:
# Funcion que entrena el arbol y luego determina la ganancia sobre el set de test
modelo_rpart_ganancia = function( ptrain, ptest, pmaxdepth, pminbucket, pminsplit, pcp )
{

  modelo   <-  rpart("clase_ternaria ~ .",   data = ptrain,  xval=0, maxdepth=pmaxdepth, minbucket=pminbucket, minsplit=pminsplit, cp=pcp)

  #aplico el modelo a datos nuevos
  testing_prediccion  <- predict( modelo, ptest)[, "BAJA+2"]
    
  return(fmetrica_ganancia_rpart(testing_prediccion,ptest[,..kclase_nomcampo],"BAJA+2"))
  
}

In [ ]:
#Cuenta los modelos probados
linea <- 1

#Establece una ganancia inicial muy baja a superar
max_gan <- -100000000

#For anidados para grid search 
for( vcp in c( 0, 0.01, 0.001)) #Complexity Factor
for( vmaxdepth in  c(4,8,14,16) ) #max Depth
for( vminsplit in  c(2, 4, 8, 10, 15, 20, 30, 50, 100, 150, 200, 300, 400 ) ) #Min Split
for( vminbucket  in  unique( as.integer(c(1,2,3,4,5,  vminsplit/10, vminsplit/5, vminsplit/3, vminsplit/2 )) ) ) #Min Bucket
{ 

    #Muestra los parametros a testear
    message(paste('Intentando',vcp,vminsplit,vminbucket,vmaxdepth))
    #Llama a la funcion que entrena el modelo y calcula la ganancia para esos parametros
    res <- modelo_rpart_ganancia( dtrain, dtest, 
                                  pmaxdepth=vmaxdepth, pminbucket=vminbucket, pminsplit=vminsplit, pcp=vcp )

  
    #Verifica si se supero la mejor ganancia hasta el momento. En caso de haberla superado graba los hiperparametros nuevos
    if (res>max_gan) {
      max_gan <- res
      message(paste("Nuevo Maximo: ", res*3, vcp,vminsplit,vminbucket,vmaxdepth))
      pcp<-vcp
      pminsplit<-vminsplit
      pminbucket<-vminbucket
      pmaxdepth<-vmaxdepth
  }
  #Muestra al ganancia de la iteracion
  message(paste('Ganancia:',res*3,'Ganancia Maxima',max_gan*3))
    
  linea <- linea+1


}

In [ ]:
#Entrena el modelo con el dataset completo
t0       <-  Sys.time()
modelo   <-  rpart("clase_ternaria ~ .",   data = dtrain,   xval=0, maxdepth=pmaxdepth, minbucket=pminbucket, minsplit=pminsplit, cp=pcp)
t1       <-  Sys.time()

tcorrida <-  as.numeric( t1 - t0, units = "secs")
print( tcorrida)

In [ ]:
#Detalles del modelo
modelo

In [ ]:
#Arbol generado
options(repr.plot.width=15, repr.plot.height=15)
prp(modelo, extra=101, digits=5, branch=1, type=4, varlen=0, faclen=0, tweak=1.3)

In [ ]:
prediccion  <- predict( modelo, dapply , type = "prob") #aplico el modelo
dapply[ , prob_baja2 := prediccion[, "BAJA+2"] ]
dapply[ , Predicted  := as.numeric(prob_baja2 > 1/40) ]
dir.create( "./exp/RPART" )
fwrite( dapply[ , list(numero_de_cliente, Predicted) ], #solo los campos para Kaggle
        file= "./exp/RPART/RPART-GRID.csv", 
        sep= "," )